# NYC Taxi data

## Import cuxfilter

In [1]:
## load cudf.pandas kernel
%load_ext cudf.pandas

In [2]:
import cuxfilter
from bokeh import palettes
from cuxfilter.layouts import feature_and_double_base
import pandas as pd
# import cudf

In [3]:
#update data_dir if you have downloaded datasets elsewhere
DATA_DIR = './'

## Download required datasets

In [3]:
! curl https://data.rapids.ai/viz-data/nyc_taxi.tar.gz --create-dirs -o $DATA_DIR/nyc_taxi.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  496M  100  496M    0     0  37.1M      0  0:00:13  0:00:13 --:--:-- 49.0M


In [4]:
from cuxfilter.sampledata import datasets_check
datasets_check('nyc_taxi', base_dir=DATA_DIR)

Dataset - .//nyc_taxi.csv

dataset already downloaded
Extracting ....
Extraction complete


## preprocess the data

In [4]:
cudf_df = pd.read_csv(DATA_DIR + '/nyc_taxi.csv').dropna()

from pyproj import Proj, Transformer

# Apply transformation
transform_4326_to_3857 = Transformer.from_crs('epsg:4326', 'epsg:3857')
cudf_df['dropoff_x'], cudf_df['dropoff_y'] = transform_4326_to_3857.transform(
                                                cudf_df['dropoff_latitude'].values_host, cudf_df['dropoff_longitude'].values_host
                                            )

cudf_df = cudf_df.drop(['dropoff_latitude', 'dropoff_longitude'], axis=1)
cudf_df = cudf_df.dropna(axis=0)

# Filter over Manhattan
cudf_df = cudf_df[
    (cudf_df.dropoff_x > -8239910.23) & (cudf_df.dropoff_x < -8229529.24) & (cudf_df.dropoff_y > 4968481.34) & (cudf_df.dropoff_y < 4983152.92)
]

cudf_df.tpep_pickup_datetime = pd.to_datetime(cudf_df.tpep_pickup_datetime, format="%Y-%m-%d")
cudf_df.tpep_dropoff_datetime = pd.to_datetime(cudf_df.tpep_dropoff_datetime, format="%Y-%m-%d")
cudf_df.head()

NameError: name 'cudf' is not defined

## Read the dataset

In [6]:
cux_df = cuxfilter.DataFrame.from_dataframe(cudf_df)

## Define charts

In [7]:
chart1 = cuxfilter.charts.scatter(x='dropoff_x',
                                 y='dropoff_y',
                                 aggregate_fn='mean',aggregate_col='payment_type', pixel_shade_type='log', legend_position='top_right',
                                 unselected_alpha=0.2,
                                 tile_provider="CartoDark", x_range=(-8239910.23,-8229529.24), y_range=(4968481.34,4983152.92))

chart2 = cuxfilter.charts.bar('passenger_count', data_points=9)
chart3 = cuxfilter.charts.bar('tpep_pickup_datetime', xrotation=45)
chart4 = cuxfilter.charts.date_range_slider('tpep_dropoff_datetime')

## Create a dashboard object

In [8]:
d = cux_df.dashboard([chart1, chart2, chart3], sidebar=[chart4], layout=feature_and_double_base, theme=cuxfilter.themes.dark, title= 'NYC TAXI DATASET')

## Starting the dashboard

1. d.show('current_notebook_url:current_notebook_port') remote dashboard

2. d.app() inline within the notebook cell

Incase you need to stop the server:

- d.stop()

In [9]:
# d.show() # notebook_url="http://localhost:8888"
d.app() # run the dashboard within the notebook cell

GridSpec(ncols=12, nrows=5)
    [0] GridSpec(height=800, ncols=12, nrows=5, sizing_mode='fixed', width=1200)
        [0] HoloViews(DynamicMap, height=320, sizing_mode='stretch_both', width=1200)
        [1] HoloViews(DynamicMap, height=480, sizing_mode='stretch_both', width=600)
        [2] HoloViews(DynamicMap, height=480, sizing_mode='stretch_both', width=600)
    [1] WidgetBox(styles={'border-color': '...})
        [0] Number(css_classes=['indicator'], default_color='#2B2B2B', font_size='18pt', format='{value:,}', name='Datapoints Selected', sizing_mode='stretch_width', title_size='14pt', value=11075631)
        [1] Progress(sizing_mode='stretch_width', styles={'--success-bg-color': '...}, value=100)
    [2] Column(min_height=500, sizing_mode='stretch_width')
        [0] DateRangeSlider(end=numpy.datetime64('2015-03-..., name='tpep_dropoff_datetime', sizing_mode='scale_width', start=numpy.datetime64('2015-01-..., styles={'color': '#2B2B2B'}, stylesheets=['\n          ...], value=(numpy.datetime64('2015-01..., value_end=numpy.datetime64('2015-03-..., value_start=numpy.datetime64('2015-01-...)

![nyc-taxi](../docs/_images/nyc-taxi.png)

## Export the queried data into a dataframe

In [ ]:
queried_df = d.export()